In [1]:
import pandas as pd
import altair as alt

In [ ]:
df = pd.read_csv('US_Accidents_March23.csv', nrows= 9999)

In [ ]:
df['Start_Time'] = pd.to_datetime(df['Start_Time'])

In [ ]:
df['Hour'] = df['Start_Time'].dt.hour
df['DayOfWeek'] = df['Start_Time'].dt.day_name()

In [ ]:
weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
weekends = ['Saturday', 'Sunday']
df['DayType'] = df['DayOfWeek'].apply(lambda x: 'Weekday' if x in weekdays else 'Weekend')

In [ ]:
hourly_data_all = df.groupby('Hour').agg({
    'ID': 'count',
    'Severity': 'mean'
}).reset_index()
hourly_data_all.columns = ['Hour', 'Crash_Count', 'Avg_Severity']
hourly_data_all['day_filter'] = 'All'

In [ ]:
hourly_data_weekdays = df[df['DayType'] == 'Weekday'].groupby('Hour').agg({
    'ID': 'count',
    'Severity': 'mean'
}).reset_index()
hourly_data_weekdays.columns = ['Hour', 'Crash_Count', 'Avg_Severity']
hourly_data_weekdays['day_filter'] = 'Weekdays'

In [ ]:
hourly_data_weekends = df[df['DayType'] == 'Weekend'].groupby('Hour').agg({
    'ID': 'count',
    'Severity': 'mean'
}).reset_index()
hourly_data_weekends.columns = ['Hour', 'Crash_Count', 'Avg_Severity']
hourly_data_weekends['day_filter'] = 'Weekends'

In [ ]:
hourly_data_days = df.groupby(['Hour', 'DayOfWeek']).agg({
    'ID': 'count',
    'Severity': 'mean'
}).reset_index()
hourly_data_days.columns = ['Hour', 'DayOfWeek', 'Crash_Count', 'Avg_Severity']
hourly_data_days = hourly_data_days.rename(columns={'DayOfWeek': 'day_filter'})

In [ ]:
combined_data = pd.concat([hourly_data_all, hourly_data_weekdays, hourly_data_weekends, hourly_data_days], ignore_index=True)

In [ ]:
stats_data = []
for filter_option in combined_data['day_filter'].unique():
    filter_data = combined_data[combined_data['day_filter'] == filter_option]
    total_crashes = filter_data['Crash_Count'].sum()
    weighted_avg_severity = (filter_data['Crash_Count'] * filter_data['Avg_Severity']).sum() / total_crashes
    stats_data.append({
        'day_filter': filter_option,
        'stat_text': f'Total Crashes: {int(total_crashes):,}  |  Overall Avg Severity: {weighted_avg_severity:.3f}'
    })

In [ ]:
stats_df = pd.DataFrame(stats_data)

In [ ]:
combined_data = combined_data.merge(stats_df, on='day_filter', how='left')

In [ ]:
day_dropdown = alt.binding_select(
    options=['All', 'Weekdays', 'Weekends', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'],
    name='Day of Week: '
)
day_selection = alt.selection_point(fields=['day_filter'], bind=day_dropdown, value='All')

base = alt.Chart(combined_data).add_params(day_selection).transform_filter(day_selection)

bars = base.mark_bar(
    opacity=0.85,
    color='#FF5582',
    cornerRadiusTopLeft=3,
    cornerRadiusTopRight=3
).encode(
    x=alt.X('Hour:O', 
            title='Hour of Day',
            axis=alt.Axis(
                labelAngle=0,
                labelFontSize=11,
                titleFontSize=13,
                titlePadding=10,
                grid=False
            )),
    y=alt.Y('Crash_Count:Q', 
            title='Frequency of Crashes',
            axis=alt.Axis(
                labelFontSize=11,
                titleFontSize=13,
                titlePadding=10,
                gridOpacity=0.3
            )),
    tooltip=[
        alt.Tooltip('Hour:O', title='Hour'),
        alt.Tooltip('Crash_Count:Q', title='Frequency of Crashes'),
        alt.Tooltip('Avg_Severity:Q', title='Avg Severity (Rated 1-4)', format='.2f')
    ]
)

line = base.mark_line(
    color='#9B59B6',
    strokeWidth=2.5,
    point=alt.OverlayMarkDef(filled=True, size=60, color='#9B59B6', opacity=0.9)
).encode(
    x=alt.X('Hour:O'),
    y=alt.Y('Avg_Severity:Q', 
            title='Avg Severity (Rated 1-4)',
            scale=alt.Scale(domain=[2.15, 2.7]),
            axis=alt.Axis(
                labelFontSize=11,
                titleFontSize=13,
                titlePadding=10,
                gridOpacity=0.3
            )),
    tooltip=[
        alt.Tooltip('Hour:O', title='Hour'),
        alt.Tooltip('Crash_Count:Q', title='Frequency of Crashes'),
        alt.Tooltip('Avg_Severity:Q', title='Avg Severity (Rated 1-4)', format='.2f')
    ]
)

main_chart = alt.layer(bars, line).resolve_scale(
    y='independent'
).properties(
    width=950,
    height=420,
    title={
        "text": "US Accidents by Hour of Day: Severity vs Frequence",
        "fontSize": 18,
        "fontWeight": 600,
        "anchor": "middle",
        "color": "#2c3e50"
    }
)

stats_text = base.mark_text(
    align='center',
    baseline='top',
    fontSize=14,
    fontWeight=500,
    color='#34495e',
    dy=10
).encode(
    x=alt.value(475),
    y=alt.value(0),
    text='stat_text:N'
).transform_aggregate(
    stat_text='max(stat_text)',
    groupby=['day_filter']
)

final_chart = alt.vconcat(
    main_chart,
    stats_text.properties(height=39)
).configure_view(
    strokeWidth=0
).configure_axis(
    labelColor='#555',
    titleColor='#333'
).properties(
    background='#fafafa'
)

final_chart